In [14]:
import json

In [15]:
import datasets
from datasets import load_dataset
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [16]:
from transformers import AutoTokenizer
from transformers import DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments, Trainer
from transformers import AutoModelForMaskedLM
from transformers import pipeline

In [17]:
model_name = 'cointegrated/rubert-tiny'

In [18]:
def start_function(examples):
    return tokenizer(examples["text"], truncation=True, max_length=312, padding='max_length')



alphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMOPQRSTUVWXYZ<>^[-]()_%*""\\\n'
num = '0123456789'
def clear_words(sent):
    new_sent = sent
    for i in new_sent:
        if i in alphabet:
            new_sent = new_sent.replace(i, '')
        elif i in num:
            return ''
    if len(new_sent) == 0:
        return ''
    if new_sent[0] == ' ':
        return new_sent[1:] 
    else:
        return new_sent

In [19]:
with open('anekdot.json', encoding="utf-8") as f:
    anecdot = json.load(f)

In [20]:
text_list = []
for i in range(len(anecdot)):
    text_list.append(anecdot[i]['text'])

In [21]:
word = [item for sentence in text_list for item in sentence.split('.') if item != '']

In [22]:
word = list(filter(lambda x: x != '\n', word))

In [23]:
new_words = []
for temp in word:
    if len(temp) > 8:
        new_temp = clear_words(temp)
        if new_temp != '':
            new_words.append(new_temp)
word = new_words

In [24]:
train_anecdot, test_anecdot = train_test_split(word, test_size=0.2)

In [25]:
train_dd = Dataset.from_dict({'text': train_anecdot})
test_dd = Dataset.from_dict({'text': test_anecdot})

In [26]:
anecdot_dd = datasets.DatasetDict({'train': train_dd, 'test': test_dd})

In [27]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [28]:
tokenizer_anecdot = anecdot_dd.map(start_function, batched=True)

  0%|          | 0/184 [00:00<?, ?ba/s]

  0%|          | 0/46 [00:00<?, ?ba/s]

In [29]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [30]:
def labels(example):
    example['labels'] = example['input_ids'].copy()
    return example

In [31]:
tokenizer_anecdot = tokenizer_anecdot.map(labels, batched=True)

  0%|          | 0/184 [00:00<?, ?ba/s]

  0%|          | 0/46 [00:00<?, ?ba/s]

In [32]:
model = AutoModelForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [33]:
model.to('cuda')

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(29564, 312, padding_idx=0)
      (position_embeddings): Embedding(512, 312)
      (token_type_embeddings): Embedding(2, 312)
      (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=312, out_features=312, bias=True)
              (key): Linear(in_features=312, out_features=312, bias=True)
              (value): Linear(in_features=312, out_features=312, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=312, out_features=312, bias=True)
              (LayerNorm): LayerNorm((312,), eps=1e-12, elementwise_affine=Tr

In [34]:
training_args = TrainingArguments(
    output_dir='D:/Anecdote_BERT/',
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=2,
    weight_decay=0.001
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenizer_anecdot["train"],
    eval_dataset=tokenizer_anecdot["test"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [35]:
import torch
torch.cuda.empty_cache()

In [36]:
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   47005 KB |   47005 KB |   47005 KB |       0 B  |\n|       from large pool |   36864 KB |   36864 KB |   36864 KB |       0 B  |\n|       from small pool |   10141 KB |   10141 KB |   10141 KB |       0 B  |\n|---------------------------------------------------------------------------|\n| Active memory         |   47005 KB |   47005 KB |   47005 KB |       0 B  |\n|       from large pool |   36864 KB |   36864 KB |

In [1]:
trainer.train()

NameError: name 'trainer' is not defined

In [ ]:
trainer.save_model('./bert_model')
bert_model = AutoModelForMaskedLM.from_pretrained('./bert_model', local_files_only=True)

fill_mask = pipeline(
    "fill-mask",
    model=bert_model,
    tokenizer=model_name,
    top_k=10
)
fill_mask('[MASK]')